# PanEntero_HIPPO: Hits (1/?)

## Imports

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2
import hippo3 as hippo
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np

## Path definitions

In [ ]:
!rm -v pan_entero.sqlite

In [ ]:
path_aligned = Path('../aligned_files')

## Create the animal

In [ ]:
animal = hippo.HIPPO('PanEntero_HIPPO','pan_entero.sqlite')

## Ingest the hits

In [ ]:
%%time
meta_df = animal.add_hits('A71EV2A', '../metadata.csv', path_aligned, skip=['A71x1148a', 'A71x1148b', 'A71x1140a', 'A71x0515a'])

In [ ]:
animal.tags.unique

In [ ]:
animal.plot_tag_statistics()

In [ ]:
animal.plot_compound_property('base')

In [ ]:
animal.plot_pose_property('target')

# test fingerprint

In [ ]:
%%time
for pose in tqdm(animal.poses):
    pose.calculate_fingerprint()

In [ ]:
animal.plot_interaction_punchcard(poses=animal.poses, subtitle='hits')

# Base compounds

In [ ]:
base_df = pd.read_excel('../base_chem.xlsx')

In [ ]:
base_df = pd.read_csv('../wt_designs_MASTER_COMPLETE_FINAL_29Feb.csv')

In [ ]:
base_df.head()

In [ ]:
from hippo3.tools import inchikey_from_smiles

for i,row in base_df.iterrows():
    
    print(row)
    
    for j in range(row['no-steps']):
        
        j += 1
        
        s1 = row[f'reactant-1-{j}']
        s2 = row[f'reactant-2-{j}']
        s3 = row[f'reaction-product-smiles-{j}']

        print(s1,s2,s3)
        
        # add the reactants
        c1 = animal.db.insert_compound(smiles=s1)
        
        if isinstance(s2,float) and np.isnan(s2):
            c2 = None
            s2 = None
        else:
            c2 = animal.db.insert_compound(smiles=s2)
            
        c3 = animal.db.insert_compound(smiles=s3)
                        
        if c1 is None:
            c1 = animal.compounds[inchikey_from_smiles(s1)]
                
        if c2 is None:
            if s2 is not None:
                c2 = animal.compounds[inchikey_from_smiles(s2)]
        
        if c3 is None:
            c3 = animal.compounds[inchikey_from_smiles(s3)]
        
        r1 = animal.db.insert_reaction(type=row[f'reaction-name-{j}'], product=c3)
        
        animal.db.insert_reactant(compound=c1, reaction=r1, amount=1)

        if c2:
            animal.db.insert_reactant(compound=c2, reaction=r1, amount=1)

        # animal.db.insert_reaction(type=row[f'reaction-name-{j}'])
        
    animal.db.insert_compound(smiles=s3)
    
    print(animal.compounds[inchikey_from_smiles(s3)])

    break

In [ ]:
recipe = animal.compounds[78].reactions[-1].get_recipe(amount=1)

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(level=logging.INFO)

recipe.draw()

In [ ]:
raise Exception

In [ ]:
meta_df[meta_df['Code'] == 'A71x0152a'].columns

In [ ]:
meta_df[meta_df['Code'] == 'A71x0152a']['[Series] Taggy McTagface'].values[0]

In [ ]:
animal.compounds

In [ ]:
animal.poses

In [ ]:
animal.poses[1].metadata
animal.poses[1].tags

In [ ]:
animal.compounds[55].mol

In [ ]:
raise Exception

In [ ]:
animal.db.query_similarity('Fc1ccc(CC2CCCNC2)cc1', 0.7, return_similarity=True)

In [ ]:
animal.compounds[38].smiles

In [ ]:
import molparse as mp

In [ ]:
mp.rdkit.mol_from_smiles('Fc1ccc(CC2CCCNC2)cc1')

In [ ]:
len(animal.db.select('compound_id',table='compound_pattern_bfp', multiple=True))

In [ ]:
len(animal.compounds)

In [ ]:
animal.db.print_table('pose')